Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed Tensorflow with Horovod
In this tutorial, you will train a word2vec model in TensorFlow using distributed training via [Horovod](https://github.com/uber/horovod).

## Prerequisites
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)
* Go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)
* Review the [tutorial](../train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb) on single-node TensorFlow training using the SDK

In [9]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.15


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [10]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [12]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Found the config file in: C:\Users\omartin\OneDrive\Professionnel\Dev\aml-sdk\MachineLearningNotebooks\aml_config\config.json
Workspace name: olitest
Azure region: eastus
Subscription id: 321cae6f-e4d3-40bf-824f-c07493a62af5
Resource group: aml-pipeline


## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [15]:
# choose a name for your cluster
cluster_name = "test-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, using it.')
except ComputeTargetException:
    print('Please create the compute target from the portal, and include (in the advanced \
           settings) the SSH public keys for Python debugging... \
           This is not yet supported via the SDK')

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target, using it.
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-20T16:14:16.323000+00:00', 'creationTime': '2019-02-19T18:52:14.624837+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-02-20T14:59:13.466856+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'targetNodeCount': 0, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}


The above code creates a GPU cluster. If you instead want to create a CPU cluster, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## Upload data to datastore
To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets. 

If your data is already stored in Azure, or you download the data as part of your training script, you will not need to do this step. For this tutorial, although you can download the data in your training script, we will demonstrate how to upload the training data to a datastore and access it during training to illustrate the datastore functionality.

First, download the training data from [here](http://mattmahoney.net/dc/text8.zip) to your local machine:

In [16]:
import os
import urllib

os.makedirs('./data', exist_ok=True)
download_url = 'http://mattmahoney.net/dc/text8.zip'
urllib.request.urlretrieve(download_url, filename='./data/text8.zip')

('./data/text8.zip', <http.client.HTTPMessage at 0x238372d2470>)

Each workspace is associated with a default datastore. In this tutorial, we will upload the training data to this default datastore.

In [17]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob olitest6851104520 azureml-blobstore-1ed2f0f5-118c-4372-a97a-e4ff3ca1e9c2


Upload the contents of the data directory to the path `./data` on the default datastore.

In [18]:
ds.upload(src_dir='data', target_path='data', overwrite=True, show_progress=True)

Uploading data\text8.zip
Uploaded data\text8.zip, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_9b1926d287ab43fda10ed7e4d72b5096

For convenience, let's get a reference to the path on the datastore with the zip file of training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--input_data` argument. 

In [19]:
path_on_datastore = 'data/text8.zip'
ds_data = ds.path(path_on_datastore)
print(ds_data)

$AZUREML_DATAREFERENCE_55b869d1b63d4c12b650911f04f83d8d


## Train model on the remote compute

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [20]:
project_folder = 'tf-distr-hvd'

In [21]:
os.makedirs(project_folder, exist_ok=True)

Copy the training script `tf_horovod_word2vec.py` into this project directory.

In [22]:
import shutil

shutil.copy('tf_horovod_word2vec.py', project_folder)

'tf-distr-hvd\\tf_horovod_word2vec.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [23]:
from azureml.core import Experiment

experiment_name = 'tf-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

### Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

In [24]:
from azureml.train.dnn import TensorFlow

In [25]:
script_params={
    '--input_data': ds_data,
    '--enable_remote_debug' :''
}

estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_horovod_word2vec.py',
                      node_count=2,
                      pip_packages = ['ptvsd==4.2.4'],
                      process_count_per_node=1,
                      distributed_backend='mpi',
                      use_gpu=True)

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, TensorFlow, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

Note that we passed our training data reference `ds_data` to our script's `--input_data` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [15]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: tf-distr-hvd,
Id: tf-distr-hvd_1550674764_27f909d3,
Type: azureml.scriptrun,
Status: Queued)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [16]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Alternatively, you can block until the script has completed training before running more code.

In [17]:
run.cancel()

In [32]:
run.wait_for_completion(show_output=True)

RunId: tf-distr-hvd_1550603179_109029bb

Streaming azureml-logs/60_control_log_rank_0.txt

This is an MPI job. Rank:0
Streaming log file azureml-logs/60_control_log_rank_0.txt
Streaming log file azureml-logs/80_driver_log_rank_0.txt

Streaming azureml-logs/80_driver_log_rank_0.txt

the input data is at /mnt/batch/tasks/shared/LS_root/jobs/olitest/azureml/tf-distr-hvd_1550603179_109029bb/mounts/workspaceblobstore/data/text8.zip
Use the data from /mnt/batch/tasks/shared/LS_root/jobs/olitest/azureml/tf-distr-hvd_1550603179_109029bb/mounts/workspaceblobstore/data/text8.zip
Found and verified /mnt/batch/tasks/shared/LS_root/jobs/olitest/azureml/tf-distr-hvd_1550603179_109029bb/mounts/workspaceblobstore/data/text8.zip
Data size 17005207
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']

{'runId': 'tf-distr-hvd_1550603179_109029bb',
 'target': 'test-compute',
 'status': 'Completed',
 'startTimeUtc': '2019-02-19T19:09:19.200602Z',
 'endTimeUtc': '2019-02-19T19:11:58.601488Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'a717f8df-2af9-4adb-b530-16b2ab794d34'},
 'runDefinition': {'Script': 'tf_horovod_word2vec.py',
  'Arguments': ['--input_data',
   '$AZUREML_DATAREFERENCE_7471201813f74114b5216eeb09e36440'],
  'SourceDirectoryDataStore': 'workspaceblobstore',
  'Framework': 0,
  'Communicator': 5,
  'Target': 'test-compute',
  'DataReferences': {'7471201813f74114b5216eeb09e36440': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': 'data/text8.zip',
    'PathOnCompute': None,
    'Overwrite': False},
   'workspaceblobstore': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment':